# Task

### Step 1: Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [10]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data set
path = r'/Users/minglunchen/Desktop/12-2023 Instacart Basket Analysis'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_derived.pkl'))

In [55]:
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,orders_interval,first_order,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag,avg_spending,spending_flag,median_orders_interval,frequency_flag
0,2539329,1,prior,1,2,8,NaN,True,196,1,...,Mid range product,Regularly busy,Regular days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,False,196,1,...,Mid range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,False,196,1,...,Mid range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,False,196,1,...,Mid range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,False,196,1,...,Mid range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


### Step 2: In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [6]:
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

### Step 3: Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

In [7]:
# Create a subset of 1000000
df_subset = ords_prods_merge[:1000000]

In [8]:
# Calculate aggregated mean of the “order_number” column grouped by “department_id” for the subset 
df_subset.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

The results for the entire dataframe are similar with those of the subset: the difference is about ±2. This minor difference is expected since a subset of the size 1000000 is quite large and relatively representative.

### Step 4: Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [34]:
# firstly create a new column with max_order
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [12]:
# then create the flag based on the criteria
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [13]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [14]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [15]:
# print the frequency of new “loyalty_flag” column 
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

### Step 5: The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [16]:
ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer      10.388747
New customer        13.294370
Regular customer    12.496203
Name: prices, dtype: float64

The prices of products purchased by loyal customers are lower than those purchased by regular or new customers.

### Step 6: The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria: 

a) If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.” 

b) If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [33]:
# firstly create a new column with avg_spending
ords_prods_merge['avg_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [24]:
# then create the flag based on the criteria
ords_prods_merge.loc[ords_prods_merge['avg_spending'] >= 10, 'spending_flag'] = 'High spender'

In [25]:
ords_prods_merge.loc[ords_prods_merge['avg_spending'] < 10, 'spending_flag'] = 'Low spender'

In [28]:
# print the frequency
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

In [27]:
# check whether right flags are assigned
ords_prods_merge[['user_id', 'spending_flag', 'avg_spending']].head(100)

,user_id,spending_flag,avg_spending
0,1,Low spender,6.367797
1,1,Low spender,6.367797
2,1,Low spender,6.367797
3,1,Low spender,6.367797
4,1,Low spender,6.367797
...,...,...,...
95,360,High spender,10.006250
96,377,Low spender,8.496552
97,387,Low spender,7.396610
98,420,Low spender,7.657813


### Step 7: In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

a) If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

b) If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

c) If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [39]:
# firstly create a new column with median of 'days_since_prior_orders' (in my case this column was renamed as 'orders_interval')
ords_prods_merge['median_orders_interval'] = ords_prods_merge.groupby(['user_id'])['orders_interval'].transform('median')

In [41]:
# then create the flag based on the criteria
ords_prods_merge.loc[ords_prods_merge['median_orders_interval'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [43]:
ords_prods_merge.loc[(ords_prods_merge['median_orders_interval'] <= 20) & (ords_prods_merge['median_orders_interval'] > 10), 'frequency_flag'] = 'Regular customer'

In [44]:
ords_prods_merge.loc[ords_prods_merge['median_orders_interval'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [45]:
# print the frequency
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [46]:
# check whether right flags are assigned
ords_prods_merge[['user_id', 'frequency_flag', 'median_orders_interval']].head(100)

,user_id,frequency_flag,median_orders_interval
0,1,Non-frequent customer,20.5
1,1,Non-frequent customer,20.5
2,1,Non-frequent customer,20.5
3,1,Non-frequent customer,20.5
4,1,Non-frequent customer,20.5
...,...,...,...
95,360,Frequent customer,4.0
96,377,Regular customer,16.5
97,387,Frequent customer,8.0
98,420,Frequent customer,7.0


### Step 9: Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [56]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_derived_grouped.pkl'))